In [12]:
import selenium
import os 
from bs4 import BeautifulSoup as bs
from urllib import request as rq
from urllib.request import urlopen
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import warnings
warnings.filterwarnings('ignore')

### 신세계 크롤링 원본 데이터 불러오기

In [2]:
shin_df = pd.read_pickle('신세계wine_delete.pkl')
shin_df

,와인이름,영어이름,한줄소개,와인타입,"나라, 와이너리",포도종,용량,곁들일 음식,당도,산도,바디,와인설명,수상,와이너리이름,와이너리설명,와이너리특징
0,이기갈 꼬뜨 뒤 론 레드,E.Guigal Cotes du Rhone Rouge,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 49%, 그르나쉬 48%, 무르베드르 3%",750ml,"차가운 육류요리나 가금류, 붉은육류요리, 치즈",당도 1,산도 2,바디 4,밝게 빛나는 진한 적색을 띠고 있으며 붉은 딸기 류의 풍부한 향과 스파이시한 노트가...,"2011 : Robert Parker 88점, Bettane + Desseauve ...",이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)..."
1,이기갈 꼬뜨 뒤 론 화이트,E.Guigal Cotes du Rhone Blanc,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",화이트 와인,프랑스 > 론 / 이 기갈,"비오니에 70%, 루산느 10%, 마르산느 9%, 끌레레뜨 8%, 부르불랑 3%",750ml,"해산물이나 매콤한 요리, 간단한 핑거푸드",당도 1,산도 3,바디 3,"비오니에 포도품종 특유의 살구, 복숭아 등의 신선한 과일 향과 향긋한 아카시아 꽃 ...",NaN,이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)..."
2,이기갈 꼬뜨 로띠 ‘브륀 & 블롱드’,E.Guigal Cot-Rotie ‘Brune & Blonde de Guigal’,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 96%, 비오니에 4%",750ml,"붉은 육류요리, 치즈",당도 1,산도 3,바디 5,"붉은 딸기, 라스베리, 체리 등의 진한 향과 오크에서 배어나는 토스트 향, 볶은 커...",2013 : Wine Spectator 93점2009 : Robert Parker ...,이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)..."
3,이기갈 꼬뜨 로띠 ‘샤또 당퓌’,E.Guigal Cote-Rotie ‘Chateau d’Ampuis’,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 93%, 비오니에 7%",750ml,"붉은 육류요리, 치즈",당도 1,산도 3,바디 5,"잘 익은 자두와 블랙베리, 라즈베리 잼처럼 진하고 응축된 과일 향과 섬세한 향신료 ...",2012 : Wine Spectator 95점 2011 : Wine Spectato...,이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)..."
4,이기갈 꽁드리유,E.Guigal Condrieu,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",화이트 와인,프랑스 > 론 / 이 기갈,비오니에 100%,750ml,"푸아그라, 송로버섯을 가미한 달걀요리 등 섬세한 풍미의 음식",당도 1,산도 3,바디 4,"살구, 복숭아, 시트러스 계열의 달콤하고 새콤한 과일 향이 주된 아로마를 이룬다. ...","2013 : Robert Parker 92점, Gault & Millau 16.5/...",이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892,까테나 자파타 까테나 알타 까베르네 소비뇽,Catena Zapata Catena Alta Cabernet Sauvignon,최적의 시기에 수확한 포도를 선별해 만든 아르헨티나 레드와인,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,까베르네 소비뇽 100%,750ml,"치킨 커틀렛, 치즈 리조또, 숙성이 잘 된 치즈",당도 1,산도 3,바디 5,까테나 알타는 까테나 가문에서 관리해 온 밭 중에서도 최상의 조화를 이룰 수 있는 ...,2019: Robert Parker 93점2018: Robert Parker 92점...,까테나 자파타 Catena Zapata,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...
893,까테나 자파타 까테나 알타 말벡,Catena Zapata Catena Alta Malbec,최적의 시기에 수확한 포도를 선별해 만든 아르헨티나 레드와인,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,말백 100%,750ml,"달콤한 육포, 스테이크, 훈연 소시지, 치즈",당도 1,산도 3,바디 4,까테나 알타는 까테나 가문에서 관리해 온 밭 중에서도 최상의 조화를 이룰 수 있는 ...,2019: Robert Parker 94점2018: James Suckling 95...,까테나 자파타 Catena Zapata,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...
894,까테나 자파타 니카시아 말벡,Catena Zapata Nicasia Malbec,까테나 자파타 최고도 빈야드에서 생산되는 100% 말벡 와인,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,말벡 100%,750ml,"양갈비, 로스트비프, 트러플 치즈 파니니, 머쉬룸 파스타",당도 1,산도 3,바디 5,"짙은 레드 계열의 바이올렛 컬러로 잘 숙성된 검은 과일, 달달한 스파이스 등 까떼나...","2018: Robert Parker 98점, James Suckling 96점201...",까테나 자파타 Catena Zapata,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...
895,까테나 자파타 아르헨티노 말벡,Catena Zapata Argentino Malbec,"열정의 나라, 아르헨티나의 프리미엄 와인",레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,말벡 100%,750ml,"치즈, 육즙이 풍부한 스테이크, 양고기, 블루 치즈",당도 1,산도 4,바디 5,"깊고 어두운 자주빛을 띠고, 라즈베리와 블랙베리의 향이 진한 와인으로 훌륭한 밸런스...","2019 James Suckling 99점, Robert Parker 96점2018...",까테나 자파타 Catena Zapata,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...


In [3]:
shin_df

,와인이름,영어이름,한줄소개,와인타입,"나라, 와이너리",포도종,용량,곁들일 음식,당도,산도,바디,와인설명,수상,와이너리이름,와이너리설명,와이너리특징
0,이기갈 꼬뜨 뒤 론 레드,E.Guigal Cotes du Rhone Rouge,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 49%, 그르나쉬 48%, 무르베드르 3%",750ml,"차가운 육류요리나 가금류, 붉은육류요리, 치즈",당도 1,산도 2,바디 4,밝게 빛나는 진한 적색을 띠고 있으며 붉은 딸기 류의 풍부한 향과 스파이시한 노트가...,"2011 : Robert Parker 88점, Bettane + Desseauve ...",이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)..."
1,이기갈 꼬뜨 뒤 론 화이트,E.Guigal Cotes du Rhone Blanc,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",화이트 와인,프랑스 > 론 / 이 기갈,"비오니에 70%, 루산느 10%, 마르산느 9%, 끌레레뜨 8%, 부르불랑 3%",750ml,"해산물이나 매콤한 요리, 간단한 핑거푸드",당도 1,산도 3,바디 3,"비오니에 포도품종 특유의 살구, 복숭아 등의 신선한 과일 향과 향긋한 아카시아 꽃 ...",NaN,이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)..."
2,이기갈 꼬뜨 로띠 ‘브륀 & 블롱드’,E.Guigal Cot-Rotie ‘Brune & Blonde de Guigal’,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 96%, 비오니에 4%",750ml,"붉은 육류요리, 치즈",당도 1,산도 3,바디 5,"붉은 딸기, 라스베리, 체리 등의 진한 향과 오크에서 배어나는 토스트 향, 볶은 커...",2013 : Wine Spectator 93점2009 : Robert Parker ...,이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)..."
3,이기갈 꼬뜨 로띠 ‘샤또 당퓌’,E.Guigal Cote-Rotie ‘Chateau d’Ampuis’,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 93%, 비오니에 7%",750ml,"붉은 육류요리, 치즈",당도 1,산도 3,바디 5,"잘 익은 자두와 블랙베리, 라즈베리 잼처럼 진하고 응축된 과일 향과 섬세한 향신료 ...",2012 : Wine Spectator 95점 2011 : Wine Spectato...,이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)..."
4,이기갈 꽁드리유,E.Guigal Condrieu,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",화이트 와인,프랑스 > 론 / 이 기갈,비오니에 100%,750ml,"푸아그라, 송로버섯을 가미한 달걀요리 등 섬세한 풍미의 음식",당도 1,산도 3,바디 4,"살구, 복숭아, 시트러스 계열의 달콤하고 새콤한 과일 향이 주된 아로마를 이룬다. ...","2013 : Robert Parker 92점, Gault & Millau 16.5/...",이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892,까테나 자파타 까테나 알타 까베르네 소비뇽,Catena Zapata Catena Alta Cabernet Sauvignon,최적의 시기에 수확한 포도를 선별해 만든 아르헨티나 레드와인,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,까베르네 소비뇽 100%,750ml,"치킨 커틀렛, 치즈 리조또, 숙성이 잘 된 치즈",당도 1,산도 3,바디 5,까테나 알타는 까테나 가문에서 관리해 온 밭 중에서도 최상의 조화를 이룰 수 있는 ...,2019: Robert Parker 93점2018: Robert Parker 92점...,까테나 자파타 Catena Zapata,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...
893,까테나 자파타 까테나 알타 말벡,Catena Zapata Catena Alta Malbec,최적의 시기에 수확한 포도를 선별해 만든 아르헨티나 레드와인,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,말백 100%,750ml,"달콤한 육포, 스테이크, 훈연 소시지, 치즈",당도 1,산도 3,바디 4,까테나 알타는 까테나 가문에서 관리해 온 밭 중에서도 최상의 조화를 이룰 수 있는 ...,2019: Robert Parker 94점2018: James Suckling 95...,까테나 자파타 Catena Zapata,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...
894,까테나 자파타 니카시아 말벡,Catena Zapata Nicasia Malbec,까테나 자파타 최고도 빈야드에서 생산되는 100% 말벡 와인,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,말벡 100%,750ml,"양갈비, 로스트비프, 트러플 치즈 파니니, 머쉬룸 파스타",당도 1,산도 3,바디 5,"짙은 레드 계열의 바이올렛 컬러로 잘 숙성된 검은 과일, 달달한 스파이스 등 까떼나...","2018: Robert Parker 98점, James Suckling 96점201...",까테나 자파타 Catena Zapata,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...
895,까테나 자파타 아르헨티노 말벡,Catena Zapata Argentino Malbec,"열정의 나라, 아르헨티나의 프리미엄 와인",레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,말벡 100%,750ml,"치즈, 육즙이 풍부한 스테이크, 양고기, 블루 치즈",당도 1,산도 4,바디 5,"깊고 어두운 자주빛을 띠고, 라즈베리와 블랙베리의 향이 진한 와인으로 훌륭한 밸런스...","2019 James Suckling 99점, Robert Parker 96점2018...",까테나 자파타 Catena Zapata,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...


# VIVINO 사이트 크롤링
-  크롤링한 신세계 와인의 <b>도수, 탄닌, flavor(아로마 향)</b> 크롤링

## TEST 크롤링

In [79]:
import re
import numpy as np

options = webdriver.ChromeOptions()
options.add_argument('window_size=1920, 1080')
driver = webdriver.Chrome('C:/cakd7/tool/chromedriver.exe', options = options)
driver.maximize_window()


url = 'https://www.vivino.com/e-guigal-cotes-du-rhone-rouge/w/7773'
driver.get(url)


body = driver.find_element(By.CSS_SELECTOR, 'body')
for i in range(6):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

time.sleep(3)


# 크롤링
## 알콜도수
dosu = driver.find_element(By.XPATH, '//*[@id="wine-page-lower-section"]/div/div[1]/div/div[2]/table/tbody').text
dosu = re.search('\d+', str(dosu))
print(dosu.group())

time.sleep(3)

## 탄닌
tannine = driver.find_elements(By.XPATH, '/html/body/div[2]/div[7]/div/div[1]/div/div[4]/div/div[2]/div[1]/table/tbody/tr')
for tan in tannine:
    try:
        if len(re.findall('Tannic',str(tan.text))) == 1:
            print(re.findall('Tannic',str(tan.text)))
            
            percent = tan.find_element(By.CLASS_NAME, 'indicatorBar__progress--3aXLX').get_attribute('style')
            print(percent)
    except:
        percent = np.nan


## flavor
flavors = driver.find_elements(By.XPATH, '/html/body/div[2]/div[7]/div/div[1]/div/div[4]/div/div[3]/div/div/div/div/button/div[2]/div[1]')

for i in range(3):
    flavors[i].click()
    F1 = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div/div[2]/div/div[2]/span').text
    print(F1)
    
    f1 = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div/div[2]/div/div[4]').text
    f1_all = f1.split('\n')
    f_w = [f1_all[i] for i in range(len(f1_all)) if i%2 == 1]
    print(f_w)
    time.sleep(2)
    
    driver.find_element(By.XPATH, '//*[@id="baseModal"]/div/div[1]/a').click()
    time.sleep(2)
    
    
driver.close()

13
['Tannic']
width: 15%; left: 47.9623%;
width: 15%; left: 47.9623%;
Black Fruit
['plum', 'blackberry', 'dark fruit', 'black fruit', 'black cherry', 'blackcurrant']
Oaky
['oak', 'vanilla', 'tobacco', 'chocolate', 'cedar', 'clove']
Earthy
['earthy', 'leather', 'smoke', 'minerals', 'mushroom', 'stone']


## 전체 크롤링

In [6]:
# 필요한 컬럼 생성

shin_df['도수'] = np.nan
shin_df['탄닌'] = np.nan
shin_df['아로마1_대분류'] = np.nan
shin_df['아로마1_소분류'] = np.nan
shin_df['아로마2_대분류'] = np.nan
shin_df['아로마2_소분류'] = np.nan
shin_df['아로마3_대분류'] = np.nan
shin_df['아로마3_소분류'] = np.nan

len(shin_df.columns)

24

In [57]:
options = webdriver.ChromeOptions()
options.add_argument('window_size=1920,1080')
driver = webdriver.Chrome('C:/cakd7/tool/chromedriver.exe', options = options)
driver.maximize_window()

# 비비노 사이트 접속
url = 'https://www.vivino.com/'
driver.get(url)


# 크롤링
for idx in range(159,len(shin_df)):
    # VIVINO 홈페이지에 신세계사이트에서 크롤링한 와인이름 검색
    wine_name = shin_df.loc[idx, '영어이름']
    search_bar = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/nav/div[1]/div/div/div/form/input')    
    search_bar.send_keys(wine_name)
    search_bar.send_keys(Keys.ENTER)
    
    # 로딩시간 (사이트 자체가 많이 느림)
    time.sleep(6)
    
    # 첫번째 게시물 클릭(가장 유사율이 높음)
    search_first = driver.find_element(By.XPATH, '/html/body/div[2]/section[1]/div/div/div/div[1]/div/div[1]/div/div[2]/div[1]/span[1]/a')
    search_first.click()
    
    time.sleep(2)
    
    
    # 와인 상세설명 페이지 크롤링
    ## 스크롤 내리기(내려야 로딩이 빨리 됨)
    body = driver.find_element(By.CSS_SELECTOR, 'body')
    for i in range(5):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    time.sleep(3)


    ## 알코올 도수: 없는 경우 많음
    try:
        dosu = driver.find_element(By.XPATH, '//*[@id="wine-page-lower-section"]/div/div[1]/div/div[2]/table/tbody').text
        dosu = re.search('\d+', str(dosu))
        shin_df.loc[idx,'도수'] = int(dosu.group())
    except:
        pass

    time.sleep(3)

    ## 탄닌
    tannine = driver.find_elements(By.XPATH, '/html/body/div[2]/div[7]/div/div[1]/div/div[4]/div/div[2]/div[1]/table/tbody/tr')
    for tan in tannine:
        try:
            if len(re.findall('Tannic', str(tan.text))) == 1:
                percent = tan.find_element(By.CLASS_NAME, 'indicatorBar__progress--3aXLX').get_attribute('style')
                shin_df.loc[idx, '탄닌'] = str(percent).split('left: ')[1].split('%')[0]
        except:
            pass


    ## flavor(아로마 향)
    try:
        flavors = driver.find_elements(By.XPATH, '/html/body/div[2]/div[7]/div/div[1]/div/div[4]/div/div[3]/div/div/div/div/button/div[2]/div[1]')
        
        # 대표 아로마향 중분류 3가지 및 소분류만 수집
        for i in range(3):
            flavors[i].click()
            F1 = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div/div[2]/div/div[2]/span').text
            shin_df.iloc[idx,-6 + (i*2)] = F1

            f1 = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div/div[2]/div/div[4]').text
            f1_all = f1.split('\n')
            f_w = ', '.join([f1_all[i] for i in range(len(f1_all)) if i%2 == 1])
            shin_df.iloc[idx,-5 + (i*2)] = f_w
            time.sleep(3)

            driver.find_element(By.XPATH, '//*[@id="baseModal"]/div/div[1]/a').click()
            time.sleep(3)    
    except:
        pass


driver.close()

In [58]:
shin_df

,와인이름,영어이름,한줄소개,와인타입,"나라, 와이너리",포도종,용량,곁들일 음식,당도,산도,...,와이너리설명,와이너리특징,도수,탄닌,아로마1_대분류,아로마1_소분류,아로마2_대분류,아로마2_소분류,아로마3_대분류,아로마3_소분류
0,이기갈 꼬뜨 뒤 론 레드,E.Guigal Cotes du Rhone Rouge,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 49%, 그르나쉬 48%, 무르베드르 3%",750ml,"차가운 육류요리나 가금류, 붉은육류요리, 치즈",당도 1,산도 2,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,47.9624,Black Fruit,"plum, blackberry, dark fruit, black fruit, bla...",Oaky,"oak, vanilla, tobacco, chocolate, cedar, clove",Earthy,"earthy, leather, smoke, minerals, mushroom, stone"
1,이기갈 꼬뜨 뒤 론 화이트,E.Guigal Cotes du Rhone Blanc,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",화이트 와인,프랑스 > 론 / 이 기갈,"비오니에 70%, 루산느 10%, 마르산느 9%, 끌레레뜨 8%, 부르불랑 3%",750ml,"해산물이나 매콤한 요리, 간단한 핑거푸드",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,NaN,Tree Fruit,"peach, apricot, pear, apple, melon, stone fruit",Earthy,"minerals, honey, stone, earthy, smoke, ginger",Citrus,"citrus, lemon, grapefruit, lime, orange, tange..."
2,이기갈 꼬뜨 로띠 ‘브륀 & 블롱드’,E.Guigal Cot-Rotie ‘Brune & Blonde de Guigal’,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 96%, 비오니에 4%",750ml,"붉은 육류요리, 치즈",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,66.6574,Oaky,"oak, vanilla, tobacco, chocolate, clove, cedar",Black Fruit,"blackberry, plum, black fruit, black cherry, d...",Earthy,"leather, smoke, earthy, mushroom, game, minerals"
3,이기갈 꼬뜨 로띠 ‘샤또 당퓌’,E.Guigal Cote-Rotie ‘Chateau d’Ampuis’,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 93%, 비오니에 7%",750ml,"붉은 육류요리, 치즈",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,66.3458,Oaky,"oak, vanilla, chocolate, tobacco, coffee, cedar",Black Fruit,"blackberry, plum, black fruit, dark fruit, cas...",Earthy,"leather, smoke, earthy, minerals, mushroom, tar"
4,이기갈 꽁드리유,E.Guigal Condrieu,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",화이트 와인,프랑스 > 론 / 이 기갈,비오니에 100%,750ml,"푸아그라, 송로버섯을 가미한 달걀요리 등 섬세한 풍미의 음식",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,NaN,Tree Fruit,"apricot, peach, pear, stone fruit, melon, apple",Earthy,"honey, stone, minerals, ginger, smoke, flint",Oaky,"vanilla, oak, butter, caramel, butterscotch, c..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892,까테나 자파타 까테나 알타 까베르네 소비뇽,Catena Zapata Catena Alta Cabernet Sauvignon,최적의 시기에 수확한 포도를 선별해 만든 아르헨티나 레드와인,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,까베르네 소비뇽 100%,750ml,"치킨 커틀렛, 치즈 리조또, 숙성이 잘 된 치즈",당도 1,산도 3,...,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...,NaN,42.9292,Black Fruit,"blackberry, plum, blackcurrant, black cherry, ...",Oaky,"oak, chocolate, mocha, caramel, vanilla, incense",Red Fruit,"raspberry, cherry, red fruit, strawberry, red ..."
893,까테나 자파타 까테나 알타 말벡,Catena Zapata Catena Alta Malbec,최적의 시기에 수확한 포도를 선별해 만든 아르헨티나 레드와인,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,말백 100%,750ml,"달콤한 육포, 스테이크, 훈연 소시지, 치즈",당도 1,산도 3,...,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...,14.0,32.8264,Black Fruit,"blackberry, plum, blueberry, black fruit, dark...",Oaky,"oak, vanilla, chocolate, tobacco, dark chocola...",Earthy,"leather, earthy, smoke, minerals, cocoa, mushroom"
894,까테나 자파타 니카시아 말벡,Catena Zapata Nicasia Malbec,까테나 자파타 최고도 빈야드에서 생산되는 100% 말벡 와인,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,말벡 100%,750ml,"양갈비, 로스트비프, 트러플 치즈 파니니, 머쉬룸 파스타",당도 1,산도 3,...,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...,14.0,27.0405,Oaky,"oak, chocolate, vanilla, tobacco, coffee, dark...",Black Fruit,"blackberry, plum, black fruit, blueberry, cass...",Earthy,"leather, cocoa, earthy, mi

In [59]:
# 저장하기

shin_df.to_csv('shin_df_vivino.csv')
shin_df.to_pickle('shin_df_vivino.pkl')

In [67]:
shin_df.head()

,와인이름,영어이름,한줄소개,와인타입,"나라, 와이너리",포도종,용량,곁들일 음식,당도,산도,...,와이너리설명,와이너리특징,도수,탄닌,아로마1_대분류,아로마1_소분류,아로마2_대분류,아로마2_소분류,아로마3_대분류,아로마3_소분류
0,이기갈 꼬뜨 뒤 론 레드,E.Guigal Cotes du Rhone Rouge,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 49%, 그르나쉬 48%, 무르베드르 3%",750ml,"차가운 육류요리나 가금류, 붉은육류요리, 치즈",당도 1,산도 2,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,47.9624,Black Fruit,"plum, blackberry, dark fruit, black fruit, bla...",Oaky,"oak, vanilla, tobacco, chocolate, cedar, clove",Earthy,"earthy, leather, smoke, minerals, mushroom, stone"
1,이기갈 꼬뜨 뒤 론 화이트,E.Guigal Cotes du Rhone Blanc,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",화이트 와인,프랑스 > 론 / 이 기갈,"비오니에 70%, 루산느 10%, 마르산느 9%, 끌레레뜨 8%, 부르불랑 3%",750ml,"해산물이나 매콤한 요리, 간단한 핑거푸드",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,NaN,Tree Fruit,"peach, apricot, pear, apple, melon, stone fruit",Earthy,"minerals, honey, stone, earthy, smoke, ginger",Citrus,"citrus, lemon, grapefruit, lime, orange, tange..."
2,이기갈 꼬뜨 로띠 ‘브륀 & 블롱드’,E.Guigal Cot-Rotie ‘Brune & Blonde de Guigal’,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 96%, 비오니에 4%",750ml,"붉은 육류요리, 치즈",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,66.6574,Oaky,"oak, vanilla, tobacco, chocolate, clove, cedar",Black Fruit,"blackberry, plum, black fruit, black cherry, d...",Earthy,"leather, smoke, earthy, mushroom, game, minerals"
3,이기갈 꼬뜨 로띠 ‘샤또 당퓌’,E.Guigal Cote-Rotie ‘Chateau d’Ampuis’,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 93%, 비오니에 7%",750ml,"붉은 육류요리, 치즈",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,66.3458,Oaky,"oak, vanilla, chocolate, tobacco, coffee, cedar",Black Fruit,"blackberry, plum, black fruit, dark fruit, cas...",Earthy,"leather, smoke, earthy, minerals, mushroom, tar"
4,이기갈 꽁드리유,E.Guigal Condrieu,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",화이트 와인,프랑스 > 론 / 이 기갈,비오니에 100%,750ml,"푸아그라, 송로버섯을 가미한 달걀요리 등 섬세한 풍미의 음식",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,NaN,Tree Fruit,"apricot, peach, pear, stone fruit, melon, apple",Earthy,"honey, stone, minerals, ginger, smoke, flint",Oaky,"vanilla, oak, butter, caramel, butterscotch, c..."


In [78]:
# 수집된 정보 확인

shin_df.isnull().sum()

와인이름          0
영어이름          0
한줄소개          0
와인타입          0
나라, 와이너리      0
포도종           0
용량            0
곁들일 음식        5
당도            0
산도            0
바디            0
와인설명          0
수상          401
와이너리이름        0
와이너리설명        0
와이너리특징        0
도수          568
탄닌          570
아로마1_대분류    416
아로마1_소분류    416
아로마2_대분류    418
아로마2_소분류    418
아로마3_대분류    426
아로마3_소분류    426
dtype: int64

In [119]:
shin_df2 = shin_df.copy()
shin_df2.dropna(subset = ['도수'], inplace = True)

In [153]:
shin_df2.isnull().sum()

와인이름          0
영어이름          0
한줄소개          0
와인타입          0
나라, 와이너리      0
포도종           0
용량            0
곁들일 음식        1
당도            0
산도            0
바디            0
와인설명          0
수상           97
와이너리이름        0
와이너리설명        0
와이너리특징        0
도수            0
탄닌          138
아로마1_대분류     44
아로마1_소분류     44
아로마2_대분류     44
아로마2_소분류     44
아로마3_대분류     44
아로마3_소분류     44
dtype: int64

In [122]:
shin_df2['산도'].value_counts()

산도 3    302
산도 2     16
산도 4     10
산도 1      1
Name: 산도, dtype: int64

In [124]:
shin_df2['바디'].value_counts()

바디 4    140
바디 5     92
바디 3     78
바디 2     19
Name: 바디, dtype: int64

In [129]:
shin_df2[['산도','바디','당도','도수']].value_counts()

산도    바디    당도    도수  
산도 3  바디 4  당도 1  13.0    60
      바디 5  당도 1  14.0    51
      바디 3  당도 1  13.0    29
      바디 5  당도 1  13.0    23
      바디 4  당도 1  12.0    15
                          ..
            당도 2  9.0      1
                  11.0     1
                  14.0     1
            당도 3  14.0     1
산도 4  바디 5  당도 1  14.0     1
Length: 76, dtype: int64

In [121]:
np.sort(shin_df2['도수'].unique())

array([ 1.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16.,
       17., 20.])